# Package import

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines
import tensorflow as tf
import gc
import time
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import DBPS_TF_NN_util as d_util
import SBPS_TF_NN_util as s_util
import MH_TF_NN_util as mh_util

# Data import

In [2]:
# dataset with 20*20, 400 pics for each label (training), 100 pics for each label (testing)
X_train = np.load('/home/user/chou/Py_BPSs_NN_TF/X_train.npy')
X_test = np.load('/home/user/chou/Py_BPSs_NN_TF/X_test.npy')
y_train = np.load('/home/user/chou/Py_BPSs_NN_TF/y_train.npy')
y_test = np.load('/home/user/chou/Py_BPSs_NN_TF/y_test.npy')

In [1]:
# data from keras, 28 * 28, each labels has different pics, check it below
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
want = [4,9]
newX_train = x_train[np.in1d(y_train, want)]
newY_train = y_train[np.in1d(y_train, want)]
newX_test = x_test[np.in1d(y_test, want)]
newY_test = y_test[np.in1d(y_test, want)]
encoder = OneHotEncoder(sparse=False)
newY_train_onehot = encoder.fit_transform(newY_train.reshape(-1, 1))
encoder = OneHotEncoder(sparse=False)
newY_test_onehot = encoder.fit_transform(newY_test.reshape(-1, 1))
print(np.shape(newX_train))
print(np.shape(newX_test))

np.save('X_train_keras49', newX_train.reshape(len(newX_train), 28*28))
np.save('Y_train_keras49', newY_train_onehot)
np.save('X_test_keras49', newX_test.reshape(len(newX_test), 28*28))
np.save('Y_test_keras49', newY_test_onehot)

In [8]:
np.shape(newX_train.reshape(len(newX_train), 28*28))

(11791, 784)

In [22]:
# training
total = 0
for i in range(10):
    want = [i]
    a = sum(np.in1d(y_train, want))
    print('num:' +str(i) + ': '+ str(a))
    total += a
print(total)

num:0: 5923
num:1: 6742
num:2: 5958
num:3: 6131
num:4: 5842
num:5: 5421
num:6: 5918
num:7: 6265
num:8: 5851
num:9: 5949
60000


In [23]:
# testing
total = 0
for i in range(10):
    want = [i]
    a = sum(np.in1d(y_test, want))
    print('num:' +str(i) + ': '+ str(a))
    total += a
print(total)

num:0: 980
num:1: 1135
num:2: 1032
num:3: 1010
num:4: 982
num:5: 892
num:6: 958
num:7: 1028
num:8: 974
num:9: 1009
10000


In [3]:
# keras data set load
X_train = np.load('/home/user/chou/Py_BPSs_NN_TF/X_train_keras49.npy')
X_test = np.load('/home/user/chou/Py_BPSs_NN_TF/X_test_keras49.npy')
y_train = np.load('/home/user/chou/Py_BPSs_NN_TF/Y_train_keras49.npy')
y_test = np.load('/home/user/chou/Py_BPSs_NN_TF/Y_test_keras49.npy')

In [4]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(11791, 784)
(1991, 784)
(11791, 2)
(1991, 2)


# Simulation
Every settings are similar to Bayesian Logistic Regression, check BLR_simulation.ipynb for more details

In [5]:
store_skip = 100
iterations = 1e07
verbose = 1e04
save_iter = 10000
burninIters = iterations /10 * 3

MH_settings = []
BPS_settings = []
SBPS_settings = []
DBPS_settings = []

In [ ]:
# ======
# don't forget to fix the input_size in SBPS_util.py and DBPS_util.py correspond to the input size of the data set
# ======
# DBPS_settings.append([runtime, totaltime, NN_DBPS.burnin_sample, NN_DBPS.stage1_prob, NN_DBPS.stage2_prob])
# SBPS_settings.append([runtime, totaltime, NN_SBPS.burnin_sample, NN_SBPS.after_burnin_storage_time, NN_SBPS.all_storage_time, NN_SBPS.theta_prior_bounce_count, NN_SBPS.bias_prior_bounce_count, NN_SBPS.likelihood_bounce_count])
# MH_settings.append([runtime, totaltime, NN_MH.burnin_sample, NN_MH.accept_count])

# DBPS
subset = 0
sto = 0
k = np.array([-2], dtype = 'f')
d = np.array([-2], dtype = 'f')
kappa = 10**k
delta = 10**d
a = time.time()
NN_DBPS = d_util.DBPS(X_train, y_train, delta, store_skip, save_iter)
NN_DBPS.DBPS_sampler(iterations, burninIters, verbose, kappa, subset , sto)
b = time.time()
runtime = b-NN_DBPS.burnin_time
totaltime = b-a
DBPS_settings.append([runtime, totaltime, NN_DBPS.burnin_sample, NN_DBPS.stage1_prob, NN_DBPS.stage2_prob])
del NN_DBPS
gc.collect()

np.save('DBPSsetting', np.array(DBPS_settings))

# SBPS
T = 10000
dt = 1
variance = 1
mini_batch = 500
ref = 10
sample_time = 1e-03

a = time.time()
NN_SBPS = s_util.SBPS(X_train, y_train, T, dt, variance, mini_batch, save_iter)
NN_SBPS.SBPS_sampler(ref, sample_time, iterations, burninIters, verbose)
b = time.time()
runtime = b-NN_SBPS.burnin_time
totaltime = b-a
SBPS_settings.append([runtime, totaltime, NN_SBPS.burnin_sample, NN_SBPS.after_burnin_storage_time, NN_SBPS.all_storage_time, NN_SBPS.theta_prior_bounce_count, NN_SBPS.bias_prior_bounce_count, NN_SBPS.likelihood_bounce_count, NN_SBPS.ref_count])
del NN_SBPS
gc.collect()
np.save('SBPSsetting', np.array(SBPS_settings))

# BPS
T = 1000000
dt = 1
variance = 1
mini_batch = len(X_train)
ref = 10
sample_time = 1e-03

a = time.time()
NN_SBPS = s_util.SBPS(X_train, y_train, T, dt, variance, mini_batch, save_iter)
NN_SBPS.SBPS_sampler(ref, sample_time, iterations, burninIters, verbose)
b = time.time()
runtime = b-NN_SBPS.burnin_time
totaltime = b-a
BPS_settings.append([runtime, totaltime, NN_SBPS.burnin_sample, NN_SBPS.after_burnin_storage_time, NN_SBPS.all_storage_time, NN_SBPS.theta_prior_bounce_count, NN_SBPS.bias_prior_bounce_count, NN_SBPS.likelihood_bounce_count, NN_SBPS.ref_count])
del NN_SBPS
gc.collect()

np.save('BPSsetting', np.array(BPS_settings))

# MH
can_sd = 0.5
a = time.time()
NN_MH = mh_util.MH(X_train, y_train,store_skip)
NN_MH.MH_sampler(can_sd, burninIters, iterations, verbose, save_iter)
b = time.time()
runtime = b-NN_MH.burnin_time
totaltime = b-a
MH_settings.append([runtime, totaltime, NN_MH.burnin_sample, NN_MH.accept_count])
del NN_MH
gc.collect()

np.save('MHsetting', np.array(MH_settings))


Current clock: 0.18308024466182174
Current counts: 10000
Current clock: 0.23759932573145792
Current counts: 20000
Current clock: 0.25493086930559
Current counts: 30000
Current clock: 0.28802619457093037
Current counts: 40000
Current clock: 0.3178923745958832
Current counts: 50000
Current clock: 0.3620578028998632
Current counts: 60000
Current clock: 0.38574251674294624
Current counts: 70000
Current clock: 0.45336423026757666
Current counts: 80000
Current clock: 0.5005491960544247
Current counts: 90000
Current clock: 0.5466468523951702
Current counts: 100000
Current clock: 0.5759937249335271
Current counts: 110000
Current clock: 0.6038659058667135
Current counts: 120000
Current clock: 0.6199034940591363
Current counts: 130000
Current clock: 0.6792161902606929
Current counts: 140000
Current clock: 0.6911706865856124
Current counts: 150000
Current clock: 0.7753529352399439
Current counts: 160000
Current clock: 0.8564776617692564
Current counts: 170000
Current clock: 0.9032564457551319
Cur